In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Flatten
import time

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the data
x_train = x_train / 255.0
x_test = x_test / 255.0

# Flatten each image to a vector with 784 values (since original MNIST images are 28x28)
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

# Define the model as specified
model = Sequential([
    Input(shape=(784,)),  # Adjusted to 784 to match MNIST image vector length
    Dense(16, activation='relu'),
    Dense(16, activation='relu'),
    Dense(16, activation='relu'),
    Dense(10, activation=None)  # Output logits
])
model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Start timing
start_time = time.time()

# Train the model
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

# End timing
training_time = time.time() - start_time

# Evaluate the model on the test set (optional, if you want to report accuracy)
loss, accuracy = model.evaluate(x_test, y_test)

# Print results
print(f"Training Time: {training_time:.2f} seconds")
print(f"Test Loss: {loss:.2f}, Test Accuracy: {accuracy:.2%}")

2024-06-08 15:33:55.330092: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-08 15:33:55.330186: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-08 15:33:55.439295: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
  94/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.2380 - loss: 2.1133

I0000 00:00:1717860850.064594     105 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1717860850.079163     105 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1498/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6905 - loss: 0.9486

W0000 00:00:1717860853.134312     106 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.6908 - loss: 0.9478 - val_accuracy: 0.9158 - val_loss: 0.2828
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9224 - loss: 0.2669 - val_accuracy: 0.9329 - val_loss: 0.2252
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9396 - loss: 0.2119 - val_accuracy: 0.9422 - val_loss: 0.1984
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9437 - loss: 0.1872 - val_accuracy: 0.9448 - val_loss: 0.1911
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9495 - loss: 0.1669 - val_accuracy: 0.9459 - val_loss: 0.1878
111/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9356 - loss: 0.2213

W0000 00:00:1717860868.258914     105 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9354 - loss: 0.2209
Training Time: 20.67 seconds
Test Loss: 0.20, Test Accuracy: 94.29%


W0000 00:00:1717860869.125840     107 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


In [2]:
import tensorflow as tf
import numpy as np
import time
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import mnist
import threading

# Load and prepare the MNIST dataset
(x, y), (x_test, y_test) = mnist.load_data()
x = np.expand_dims(x, -1) / 255.0  # Normalize and add channel dimension
x_resized = tf.image.resize(x, [16, 16])  # Resize images to 16x16
x_resized = x_resized[..., 0]  # Remove the channels dimension
x_flattened = tf.reshape(x_resized, [x_resized.shape[0], -1])  # Flatten images

x_test = np.expand_dims(x_test, -1) / 255.0
x_test_resized = tf.image.resize(x_test, [16, 16])
x_test_resized = x_test_resized[..., 0]
x_test_flattened = tf.reshape(x_test_resized, [x_test_resized.shape[0], -1])

x_flattened1 = x_flattened[:, :128]  # First half of the pixels
x_flattened2 = x_flattened[:, 128:]  # Second half of the pixels
x_test_flattened1 = x_test_flattened[:, :128]
x_test_flattened2 = x_test_flattened[:, 128:]

models = {}  # Dictionary to store models

def create_and_train_model(gpu_id, data, labels):
    """Create and train a model on a specific GPU."""
    with tf.device(f'/gpu:{gpu_id}'):
        model = Sequential([
            Input(shape=(128,)),
            Dense(16, activation='relu'),
            Dense(16, activation='relu'),
            Dense(16, activation='relu'),
            Dense(10, activation='softmax')
        ])
        model.compile(optimizer='adam', 
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                      metrics=['accuracy'])
        print(f"Starting training on GPU {gpu_id}")
        model.fit(data, labels, epochs=5, batch_size=32, validation_split=0.2)
        models[gpu_id] = model  # Save the model to the global dictionary
        print(f"Training completed on GPU {gpu_id}")

def run_on_gpu(gpu_id, data, labels):
    """Function to run training on a specific GPU."""
    create_and_train_model(gpu_id, data, labels)

# Setup threading to train on both GPUs concurrently
thread1 = threading.Thread(target=run_on_gpu, args=(0, x_flattened1, y))
thread2 = threading.Thread(target=run_on_gpu, args=(1, x_flattened2, y))

start_time = time.time()
thread1.start()
thread2.start()
thread1.join()
thread2.join()
end_time = time.time()

# Output the total time taken
print(f"Total elapsed time: {end_time - start_time:.4f} seconds")

Starting training on GPU 0
Starting training on GPU 1
Epoch 1/5
Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.5321 - loss: 1.3576 - val_accuracy: 0.8235 - val_loss: 0.5562
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.5711 - loss: 1.3065 - val_accuracy: 0.7843 - val_loss: 0.6252
Epoch 2/5
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7943 - loss: 0.6148 - val_accuracy: 0.8255 - val_loss: 0.5119
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8325 - loss: 0.5361 - val_accuracy: 0.8600 - val_loss: 0.4568
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8181 - loss: 0.5338 - val_accuracy: 0.8267 - val_loss: 0.4789
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8549 - loss: 0.4638 - val_accuracy: 0.8704 - val_loss: 0.4130
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8281 - loss: 0.5006 - val_accuracy: 0.8450 - val_loss: 0.4428
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━

In [3]:
def combine_predictions(model1, model2, data1, data2):
    """Combine predictions from two models by averaging their output probabilities."""
    # Ensure predictions are made on the correct GPUs
    with tf.device('/gpu:0'):
        preds1 = model1.predict(data1)
    with tf.device('/gpu:1'):
        preds2 = model2.predict(data2)
    
    # Average the predictions from both models
    combined_preds = (preds1 + preds2) / 2
    return combined_preds

def calculate_accuracy(predictions, labels):
    """Calculate the accuracy of predictions."""
    predicted_labels = np.argmax(predictions, axis=1)
    correct_predictions = np.sum(predicted_labels == labels)
    accuracy = correct_predictions / len(labels)
    return accuracy

# Use the `models` dictionary which stores trained models
combined_preds = combine_predictions(models[0], models[1], x_test_flattened1, x_test_flattened2)

# Compute probabilities using softmax and calculate accuracy
probabilities = tf.nn.softmax(combined_preds, axis=1)
final_accuracy = calculate_accuracy(probabilities, y_test)  # Ensure y_test is your actual test labels
print(f"Combined Model Accuracy: {final_accuracy:.4%}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Combined Model Accuracy: 93.5200%
